In [1]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad  # Import the pad and unpad functions
import pandas as pd
 


In [2]:


def get_vi_key(byte_size = 16):

    e_key =  get_random_bytes(byte_size)
    cipher = AES.new(e_key, AES.MODE_CBC)
    return e_key ,  cipher
    




def encrypt_data(e_key,e_cipher,pt_data):

    r_cipher_text = e_cipher.encrypt(pad(pt_data.encode('utf-8'), AES.block_size))  # Ensure data is in bytes

    return r_cipher_text



def decrypt_data(e_cipher_text, e_key, e_iv):

    decrypt_cipher = AES.new(e_key, AES.MODE_CBC, e_iv)
    plain_text = unpad(decrypt_cipher.decrypt(e_cipher_text), AES.block_size).decode('utf-8') # Decode to string
    
    return str(plain_text)


def get_random_Key_Iv_df(arrSize=5):

    keyArr=[]
    cipherArr=[]

    for i in range(arrSize):
        key, cipher = get_vi_key()
        keyArr.append(key)
        cipherArr.append(cipher)
    return keyArr,cipherArr



In [3]:


# Specify input CSV file path
csv_path = 'dataset/text.csv'
output_csv_path = 'dataset/dataframe.csv'

df = pd.read_csv(csv_path)

print(df)


                                                   text
0                                                   Hi!
1                        What is your favorite holiday?
2     one where I get to meet lots of different people.
3     What was the most number of people you have ev...
4                       Hard to keep a count. Maybe 25.
...                                                 ...
5095  you're wearing out your right hand. stop using...
5096  but i do all my writing with my right hand.sta...
5097  making this game is fun but i would like to in...
5098  In this example, a new column named 'City' is ...
5099  You can adjust the values based on your requir...

[5100 rows x 1 columns]


In [4]:


# 250 on same array of 10 variations of (vi and key)
# every 250 text values after new array 10 of key and vi
# making a new Data set for the encriptio 
data = {
    'text': [],
    'cipher_text': [],
    'key': [],
    'iv': [],
}

# Create a DataFrame with existing data
newDF = pd.DataFrame(data )

i=0
keyArr , cipherArr =  get_random_Key_Iv_df()

for txt in df['text']:
    
    if( (i%250) == 0):
        keyArr , cipherArr =  get_random_Key_Iv_df()
        print(i)

    for i in range(len(keyArr)):
        
        cipher_text = encrypt_data(keyArr[i],cipherArr[i],txt)
        newDF.loc[len(newDF.index)] = [txt, str(cipher_text), str(keyArr[i]), str(cipherArr[i].iv)]


        
    i += 1
 

newDF.to_csv('my_data.csv', index=False, encoding='utf-8')  # Set index=False to exclude the index column



0


In [8]:
import ast

csv_path = 'my_data.csv'


df = pd.read_csv(csv_path ,encoding='utf-8')

# print(type("HI!"))
 
# print(decrypt_data( ast.literal_eval(df['cipher_text'][1]) , ast.literal_eval(df['key'][1]) , ast.literal_eval(df['iv'][1] )))





In [9]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [10]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  

tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Dense, Embedding, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from scipy.sparse import hstack

# Use 'text' column as the target variable
y = df['text'].values

# Use the rest of the columns as input features
X_df = df.drop('text', axis=1)

# One-hot encode categorical columns
categorical_cols = X_df.select_dtypes(include='object').columns
encoder = OneHotEncoder(drop='first', sparse=True)
X_encoded = encoder.fit_transform(X_df[categorical_cols])

# Convert X_encoded to DataFrame if needed
X_encoded_df = pd.DataFrame.sparse.from_spmatrix(X_encoded, columns=encoder.get_feature_names_out(categorical_cols))

# Combine encoded categorical features with numerical features
X_combined = hstack([X_encoded, X_df.drop(categorical_cols, axis=1)], format='csr')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=42
)

# Build the model
embedding_dim = 50

# Input layers
input_features = [Input(shape=(1,), dtype=tf.float32) for _ in range(X_combined.shape[1])]

# Embedding layer for each input feature
embedding_layers = [Embedding(
    input_dim=max(1, int(X_combined[:, i].max()) + 1), output_dim=embedding_dim, input_length=1
)(input_feature) for i, input_feature in enumerate(input_features)]

# Concatenate all layers
concatenated_layers = Concatenate()(embedding_layers)

# Flatten layer
flattened_layers = Flatten()(concatenated_layers)

# Dense layer for prediction
output_layer = Dense(1, activation='sigmoid')(flattened_layers)

# Create the model
model = Model(inputs=input_features, outputs=output_layer)

# Convert string labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train[:, i].A.reshape(-1, 1).astype(float) for i in range(X_combined.shape[1])], y_train_encoded, epochs=20, batch_size=64, validation_split=0.2)

# Evaluate the model
test_inputs = [X_test[:, i].A.reshape(-1, 1).astype(float) for i in range(X_combined.shape[1])]
loss, accuracy = model.evaluate(test_inputs, label_encoder.transform(y_test.ravel()))
print(f'Test Accuracy: {accuracy}')



Epoch 1/5
51/51 [==============================] - 971s 6s/step - loss: -810772.8125 - accuracy: 0.0000e+00 - val_loss: -2420387.0000 - val_accuracy: 0.0012
Epoch 2/5
51/51 [==============================] - 237s 5s/step - loss: -6002678.5000 - accuracy: 0.0000e+00 - val_loss: -10884283.0000 - val_accuracy: 0.0012
Epoch 3/5
51/51 [==============================] - 219s 4s/step - loss: -18053420.0000 - accuracy: 0.0000e+00 - val_loss: -26694694.0000 - val_accuracy: 0.0012
Epoch 4/5
51/51 [==============================] - 236s 5s/step - loss: -37556704.0000 - accuracy: 0.0000e+00 - val_loss: -50005108.0000 - val_accuracy: 0.0012
Epoch 5/5
14/51 [=======>......................] - ETA: 2:40 - loss: -53522712.0000 - accuracy: 0.0000e+00

KeyboardInterrupt: 